In [36]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np

In [37]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
torch.manual_seed(1337)

In [38]:
stoi = {
    ' ': 0,
    'a': 1,
    'b': 2,
    'c': 3,
    'd': 4,
    'e': 5,
    'f': 6,
    'g': 7,
    'h': 8,
    'i': 9,
    'j': 10,
    'k': 11,
    'l': 12,
    'm': 13,
    'n': 14,
    'o': 15,
    'p': 16,
    'q': 17,
    'r': 18,
    's': 19,
    't': 20,
    'u': 21,
    'v': 22,
    'w': 23,
    'x': 24,
    'y': 25,
    'z': 26,
    '#': 27,
    '$': 28,
    '@': 29,
    '%': 30
}
# '# = is to'
# '$ = as'
# '@ = end token'
# '% = padding at the start to make it block size length'
vocab_size = len(stoi)

itos = {v: k for k, v in stoi.items()}

# def tokenize(text):
#     return [CHAR_TO_TOKEN[c] for c in text]

# def detokenize(tokens):
#     return ''.join([TOKEN_TO_CHAR[t] for t in tokens])

encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string


In [39]:
word_pairs = []
with open('nouns.csv', 'r') as f:
    for line in f:
        parts = line.strip().split(',')
        if len(parts) >= 2:
            singular = parts[0].strip()
            plural = parts[1].strip()

            if all(c in stoi for c in singular) and all(c in stoi for c in plural):
                word_pairs.append((singular, plural))

print(word_pairs[0])

('aab', 'aabs')


In [40]:
n = int(0.9*len(word_pairs)) # first 90% will be train, rest val
train_data = word_pairs[:n]
val_data = word_pairs[n:]

In [44]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.zeros((batch_size, block_size), dtype=int)
    y = torch.zeros((batch_size, block_size), dtype=int)

    for i in range(batch_size):
        n = len(data)
        s1, p1 = data[np.random.randint(n)]
        s2, p2 = data[np.random.randint(n)]
        window = np.random.randint(len(p2))
        p2 += '@'

        '''
        Data is of the form:
        %%%% singular $ plural # singular $ part of a plural

        We pad with %%% to keep the size correct

        For example:
        x = %%%%%%%fox$foxes#dog$do
        y = %%%%%%fox$foxes#dog$dog
        '''


        xi = (s1 + '#' + p1 + '$' + s2 + '#' + p2[:window]).rjust(block_size, '%')
        yi = (s1 + '#' + p1 + '$' + s2 + '#' + p2[:window + 1]).rjust(block_size, '%')
        x[i] = torch.tensor(encode(xi))
        y[i] = torch.tensor(encode(yi))

    x, y = x.to(device), y.to(device)
    return x, y

# batch = get_batch('train')
# print(batch[0].shape)
# print(batch[1].shape)

torch.Size([64, 256])
torch.Size([64, 256])


In [45]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

    def evaluate(model, word_pairs, stoi, itos, k=3, max_len=128):
        model.eval()
        correct = 0
        total = 20
        with torch.no_grad():
            for _ in range(total):
                context, target = create_context_examples(word_pairs, k=k)
                input_ids = tokenize(context, stoi)
                input_tensor = torch.tensor(input_ids + [0] * (max_len - len(input_ids)), dtype=torch.long)[:max_len].unsqueeze(0)

                output_logits = model(input_tensor)
                output_ids = output_logits.argmax(dim=-1)[0].tolist()

                predicted = detokenize(output_ids[len(input_ids):], itos)
                print(f"{context}{predicted.strip()} (Expected: {target})")
                if predicted.strip().startswith(target):
                    correct += 1

        print(f"Accuracy: {correct}/{total}")
        model.train()

In [46]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

10.762783 M parameters
step 0: train loss 3.3831, val loss 3.3779
step 500: train loss 0.3910, val loss 0.4115
step 1000: train loss 0.2230, val loss 0.2473
step 1500: train loss 0.2174, val loss 0.2449
step 2000: train loss 0.2050, val loss 0.2491
step 2500: train loss 0.2007, val loss 0.2551
step 3000: train loss 0.1918, val loss 0.2402
step 3500: train loss 0.1888, val loss 0.2384
step 4000: train loss 0.1844, val loss 0.2338
step 4500: train loss 0.1814, val loss 0.2297
step 4999: train loss 0.1787, val loss 0.2284
 %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%i%tholphome#iirholphomes$each grefloweel#each grefloweels$bosel#each grefloweelsels$green#goseacheelseelss oap#itelosess os$llevulettory#elecoe cerlist#eellies olders measet#erle veale menitar meneral signet#eea mencen ases$dosper#erane dospers$dede#elenten coeas$ado